In [1]:
from nltk.corpus import stopwords
import watson_nlp
from watson_core import data_model as dm
import nltk
from nltk.corpus import stopwords as sw
import glob
import re
import json
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import os

sw = stopwords.words('english')

ModuleNotFoundError: No module named 'watson_nlp'

In [ ]:
filepath = "data/afghan_small.csv"
data = pd.read_csv(filepath,low_memory=False)

In [ ]:
data['text'] = data['text'].apply(lambda x: str(x).lower())

In [ ]:
data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]','',str(x).lower()))

In [ ]:
data.head()

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
        return True
    except UnicodeDecodeError:
        return False
    else:
        pass
#checks if the text is in English

In [ ]:
count = 0
eng_count = []
for idx, row in data.iterrows():
    eng_count.append(isEnglish(row['text'])) 
    if isEnglish(row['text']) == False:
        count += 1
#appends the result of isEnglish to a list and counts the number of English words per row

In [ ]:
data['eng'] = eng_count
# adds the result of isEnglish to a column in the dataframe, true if in English false otherwise

In [ ]:
data = data[(data['eng'] == True)]
#filters out all tweets that aren't in English

In [ ]:
len(data['created_at'])

In [ ]:
from datetime import datetime

new_datetime = datetime.strftime(datetime.strptime(data['created_at'][0],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
# converts timestamp into UTC format

In [ ]:
new_datetime

In [ ]:
dt_list = []
for i in data['created_at']:
    try:
        dt_list.append(datetime.strftime(datetime.strptime(i,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S'))
    except:
        dt_list.append(dt_list[0])
    #creates a list with timestamps in the new format

In [ ]:
data['created_at_datetime'] = dt_list
#generates a new column in the dataframe identical to the list

In [ ]:
data['created_at_datetime'] = pd.to_datetime(data['created_at_datetime'])
#changes column to datetime format

In [ ]:
data['created_at_datetime'] = data['created_at_datetime'].sort_values()
#sorts the column by the day the tweet was sent/created

In [ ]:
data = data.sort_values(by='created_at_datetime').reset_index()
#orders entire dataframe according to the time in which the tweet was created

In [ ]:
data.shape

In [ ]:
u = data["created_at_datetime"].map(lambda t: t.date()).unique()

In [ ]:
u

In [ ]:
type(u[0])

In [ ]:
data['created_at_date'] = data['created_at_datetime'].apply(lambda x: x.date())

In [ ]:
data['created_at_datetime'] = pd.to_datetime(data['created_at_datetime'])

In [ ]:
filter_mask = data['created_at_date'] == u[-1]
filtered_df = data[filter_mask]

In [ ]:
filter_mask

In [ ]:
def rename_csv_filenames(output_file):
    for filepath in glob.iglob(f'{output_file}/*.csv'):
        os.rename(filepath ,output_file + '/' + 'docs.csv')

In [ ]:
train_params = {
    'min_words_per_utterance': 10,
    'num_turns_to_remove': 0,
    'beginning_ratio': 1,
    'beginning_weighting_factor': 1,
    'min_ngram_size': 4,
    'max_ngram_size': 12,
    'max_ngrams': 12,
    'stopwords': list(sw) #nltk stopwords
}
#establishing the training paremeters

In [ ]:
train_params_hierarchical = {
    'king_cluster_min_ratio': .2,
    'min_records_per_king_cluster': 1000,
    'num_topics_per_iteration': 10,
    'max_num_iters_per_model': 10,
    'min_word_support': 0.001,
    'max_word_support': 0.7,
    'max_ngrams_per_topic': 12,
}
#limits for said parameters

In [ ]:
output_file = 'out.csv'
filename = "data/spark_format.csv"
final_docs = 'out.csv/docs.csv'
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_en_stock','matthew.huntebrinker@ibm.com','AKCp8ihpbQ2esQQLh1BpaZKFAgDpieBx2UzN674kreY4jnNMLCJXW5jdXmmbWD7jzowoegoZ7'))
for dt in u:
    tmp_df = data[data['created_at_date']==dt]
    df2 = pd.DataFrame()
    df2['json'] = tmp_df['text']
    df2['id'] = df2.index
    df2 = df2[['id', 'json']]
    df2.to_csv('data/spark_format.csv',index=False)
    spark = SparkSession.builder.appName("readdata").getOrCreate()
    df = spark.read.csv(filename, inferSchema=True, header=True)
    df.coalesce(1).write.option("header","true").option("sep","|").mode("overwrite").csv(output_file)
    rename_csv_filenames(output_file)
    csv_stream = dm.DataStream.from_csv(final_docs, skip=1, delimiter='|')
    syntax_data2 = syntax_model.stream(csv_stream[1])
    summary_model2 = watson_nlp.toolkit.NGramSummary.train(train_data=syntax_data2,
                                                     train_params=train_params)
    topic_model2 = watson_nlp.blocks.topics.HierarchicalClustering.train(train_data=syntax_data2, 
                                                                    summary_model=summary_model2,
                                                                    train_params=train_params_hierarchical)
    with open(f'data/markdown/afghan_topic_modeling_{dt}.csv.md', 'w') as f:
        f.write(topic_model2.model.to_markdown())
    topic_model2.save(f"models/model_{dt}")

In [ ]:
df2 = pd.DataFrame()

In [ ]:
df2['json'] = data['text']

In [ ]:
df2['id'] = df2.index

In [ ]:
df2 = df2[['id', 'json']]

In [ ]:
df2.to_csv('data/spark_format.csv',index=False)

In [ ]:
df2.head()

In [ ]:
# with open('creds.json') as config:
#     creds = json.load(config)

In [ ]:
output_file = 'out.csv'

In [ ]:
spark = SparkSession.builder.appName("readdata").getOrCreate()
filename = "data/spark_format.csv"
df = spark.read.csv(filename, inferSchema=True, header=True)

In [ ]:
df.coalesce(1).write.option("header","true").option("sep","|").mode("overwrite").csv(output_file)

In [ ]:

def rename_csv_filenames(output_file):
    for filepath in glob.iglob(f'{output_file}/*.csv'):
        print(filepath)
        os.rename(filepath ,output_file + '/' + 'docs.csv')

In [ ]:
rename_csv_filenames(output_file)

In [ ]:
final_docs = 'out.csv/docs.csv'

In [ ]:
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_en_stock','matthew.huntebrinker@ibm.com','AKCp8ihpbQ2esQQLh1BpaZKFAgDpieBx2UzN674kreY4jnNMLCJXW5jdXmmbWD7jzowoegoZ7'))

In [ ]:
csv_stream = dm.DataStream.from_csv(final_docs, skip=1, delimiter='|')

In [ ]:
# for idx, row in enumerate(csv_stream):
#     print(row)
#     if idx == 4:
#         break

In [ ]:
syntax_data2 = syntax_model.stream(csv_stream[1])

In [ ]:
topic_model_2_prediction = topic_model2.run_batch(syntax_data2)

In [ ]:
print('done!')

In [ ]:
print('done done')

In [ ]:
print('d')

In [ ]:
print(type(topic_model_2_prediction))

In [ ]:
# count = 0
# for i in syntax_data2:
#     count+=1
#     if count > 5:
#         break
#     print(i)

In [ ]:
# get_topics

In [ ]:
# man_words = ['to']

In [ ]:
train_params = {
    'min_words_per_utterance': 10,
    'num_turns_to_remove': 0,
    'beginning_ratio': 1,
    'beginning_weighting_factor': 1,
    'min_ngram_size': 4,
    'max_ngram_size': 12,
    'max_ngrams': 12,
    'stopwords': list(sw) #nltk stopwords
}

In [ ]:
summary_model2 = watson_nlp.toolkit.NGramSummary.train(train_data=syntax_data2,
                                                     train_params=train_params)
#figure out why stopwords didn't work

In [ ]:
# summary_model2.model.get_stop_words()

In [ ]:
train_params_hierarchical = {
    'king_cluster_min_ratio': .2,
    'min_records_per_king_cluster': 1000,
    'num_topics_per_iteration': 10,
    'max_num_iters_per_model': 10,
    'min_word_support': 0.001,
    'max_word_support': 0.7,
    'max_ngrams_per_topic': 12,
}

In [ ]:
topic_model2 = watson_nlp.blocks.topics.HierarchicalClustering.train(train_data=syntax_data2, 
                                                                    summary_model=summary_model2,
                                                                    train_params=train_params_hierarchical)

In [ ]:
with open('data/markdown/afghan_topic_modeling12.csv.md', 'w') as f:
    f.write(topic_model2.model.to_markdown())

In [ ]:
len(topic_model2.get_topics())

In [ ]:
s = 0
for i in topic_model2.get_topics():
    s += i.size
print(s)

In [ ]:
# topic_model2.get_topics()

In [ ]:
topic_model2.save("models/model")

In [ ]:
summary_model2.save("models/syntax.model")

In [ ]:
matt = data['text'].iloc[0]

In [ ]:
f = syntax_model.run(matt)

In [ ]:
fi = topic_model2.run(f)

In [ ]:
# for idx, row in data.iterrows():
#     f = syntax_model.run(str(row['text']))
#     fi = topic_model2.run(f)
#     print(fi)
#     if idx == 50:
#         break

In [ ]:
topic_model2 = watson_nlp.load('models/model')

In [ ]:
topic_model2.to_md()

In [ ]:
len(topics)

In [ ]:
len(data['text'])

In [ ]:
data['topics'] = topics

In [ ]:
topics = []
count = 0
for x in data['text']:
    f = syntax_model.run(x)
    topics.append(topic_model2.run(f))
    count += 1
    if count % 5000 == 0:
        print(count)

In [ ]:
type(data['text'].tolist())

In [ ]:
f = syntax_model.run(data['text'].tolist())

In [ ]:
run_batch

In [ ]:
data['topics'] = topics

In [ ]:
data.to_csv('data/topics.csv',index=False)

In [ ]:
import ast

In [ ]:
data['topics'].head(50).iloc[0].topics

In [ ]:
topics_name = []
for idx, i in enumerate(data['topics']):
    if not i.topics:
        topics_name.append('None')
    elif i.topics[0].name:
        topics_name.append(i.topics[0].name)

In [ ]:
len(topics_name)

In [ ]:
data['topics_name'] = topics_name

In [ ]:
len(topics)

In [ ]:
data.to_csv('data/topic_names.csv',index=False)

In [ ]:
data_topics = data[data['topics_name'] != 'None']

In [ ]:
list(set(data_topics['topics_name']))

In [ ]:
import json
import pandas as pd
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions, EntitiesOptions, KeywordsOptions

authenticator = IAMAuthenticator('Jeg4w84k1cdxcmnsiTk6s8CKAWHpl5zVLBCxCpoWvE80')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2021-03-25',
    authenticator=authenticator)
natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/7bd1aff3-0d4b-4ee3-974e-013c8befa581')

In [ ]:
def parse_nlu_results(text):
    response = natural_language_understanding.analyze(
        text=text,
        language="en",
        features=Features(sentiment=SentimentOptions(), emotion=EmotionOptions(), 
                          entities=EntitiesOptions(emotion=True, sentiment=True),
                          keywords=KeywordsOptions(emotion=True, sentiment=True),
                         )).get_result()
    
    # seentiment
    try:
        score = response.get('sentiment').get("document").get("score")
        label = response.get('sentiment').get("document").get("label")
    except:
        score = ""
        label = ""
    
    # emotions
    try:
        sadness = response.get("emotion").get("document").get("emotion").get("sadness")
        joy = response.get("emotion").get("document").get("emotion").get("joy")
        fear = response.get("emotion").get("document").get("emotion").get("fear")
        anger = response.get("emotion").get("document").get("emotion").get("anger")
        disgust = response.get("emotion").get("document").get("emotion").get("disgust")
    except:
        sadness = ""
        joy = ""
        fear = ""
        anger = ""
        disgust = ""
    
    # keywords
    try:
        keywords = response.get("keywords")
    except:
        keywords = ""
    
    # entities
    try:
        entities = response.get("entities")
    except:
        entities = ""
    
    return score, label, sadness, joy, fear, anger, disgust, keywords, entities

In [ ]:
nlu_results = []

for idx, x in enumerate(data['text']):
    score, label, sadness, joy, fear, anger, disgust, keywords, entities = parse_nlu_results(x)
    nlu_results.append((score, label, sadness, joy, fear, anger, disgust, keywords, entities))
    if idx % 2000 == 0:
        print(idx)